# Section 6: Perform a Simple EDA in R

### All code below is referenced from Lecture_1_2.ipynb provided by Gittu George for DSCI 525

In [1]:
import dask.dataframe as dd
import re
import os
import glob
import sys
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np
from memory_profiler import memory_usage
from os import listdir

import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pyarrow.feather as feather
import rpy2_arrow.pyarrow_rarrow as pyra

In [2]:
%load_ext memory_profiler


In [3]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


### Method 1: Pandas Exchange

In [11]:
%%time
%%memit

pd_exchange_df = pd.read_csv("figshareairline/combo_data.csv", index_col = 0)

/Users/raf/opt/miniconda3/envs/525/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


peak memory: 16599.21 MiB, increment: 596.06 MiB
CPU times: user 1min 7s, sys: 18.3 s, total: 1min 25s
Wall time: 1min 34s


In [13]:
f = open("figshareairline/combo_data.csv", "r")
print(f.readlines(5))

[',model,time,lat_min,lat_max,lon_min,lon_min,rain (mm/day)\n']


In [15]:
f = open("figshareairline/combo_data.csv", "r")
print(f.readlines(5))

['model,time,lat_min,lat_max,lon_min,lon_min.1,rain (mm/day)\n']


In [12]:
pd_exchange_df.head()

,model,time,lat_min,lat_max,lon_min,lon_min.1,rain (mm/day)
0,MPI-ESM-1-2-HAM,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,141.5625,4.244226e-13
1,MPI-ESM-1-2-HAM,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,141.5625,4.217326e-13
2,MPI-ESM-1-2-HAM,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,141.5625,4.498125e-13
3,MPI-ESM-1-2-HAM,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,141.5625,4.251282e-13
4,MPI-ESM-1-2-HAM,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,141.5625,4.270161e-13


In [14]:
pd_exchange_df.to_csv("figshareairline/combo_data.csv", index=False)

In [10]:
pd.read_csv("figshareairline/combo_data.csv", index_col = 0, nrows=5)

,model,time,lat_min,lat_max,lon_min,lon_min.1,rain (mm/day)
0,MPI-ESM-1-2-HAM,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,141.5625,4.244226e-13
1,MPI-ESM-1-2-HAM,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,141.5625,4.217326e-13
2,MPI-ESM-1-2-HAM,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,141.5625,4.498125e-13
3,MPI-ESM-1-2-HAM,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,141.5625,4.251282e-13
4,MPI-ESM-1-2-HAM,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,141.5625,4.270161e-13


In [8]:
pd_exchange_df[pd_exchange_df['model'] == "GFDL-ESM4"].head()

,Unnamed: 0,model,time,lat_min,lat_max,lon_min,lon_min.1,rain (mm/day)
55155510,0,GFDL-ESM4,1889-01-01 12:00:00,NaN,-36.0,NaN,NaN,NaN
55155511,1,GFDL-ESM4,1889-01-02 12:00:00,NaN,-36.0,NaN,NaN,NaN
55155512,2,GFDL-ESM4,1889-01-03 12:00:00,NaN,-36.0,NaN,NaN,NaN
55155513,3,GFDL-ESM4,1889-01-04 12:00:00,NaN,-36.0,NaN,NaN,NaN
55155514,4,GFDL-ESM4,1889-01-05 12:00:00,NaN,-36.0,NaN,NaN,NaN


In [6]:
%%R

library(dplyr)
library(arrow)
library(lubridate)

R[write to console]: trying URL 'http://cran.us.r-project.org/src/contrib/lubridate_1.7.10.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 473547 bytes (462 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to 

In [7]:
%%time
%%R -i pd_exchange_df

start_time <- Sys.time()
result <- pd_exchange_df %>% count(model)
print(result)
end_time <- Sys.time()

print(end_time - start_time)

              model       n
1        ACCESS-CM2 1932840
2     ACCESS-ESM1-5 1610700
3    AWI-ESM-1-1-LR  966420
4       BCC-CSM2-MR 3035340
5          BCC-ESM1  551880
6           CanESM5  551880
7      CMCC-CM2-HR4 3541230
8      CMCC-CM2-SR5 3541230
9         CMCC-ESM2 3541230
10 EC-Earth3-Veg-LR 3037320
11      FGOALS-f3-L 3219300
12        FGOALS-g3 1287720
13         GFDL-CM4 3219300
14        GFDL-ESM4 3219300
15        INM-CM4-8 1609650
16        INM-CM5-0 1609650
17        KIOST-ESM 1287720
18           MIROC6 2070900
19  MPI-ESM-1-2-HAM  966420
20    MPI-ESM1-2-HR 5154240
21    MPI-ESM1-2-LR  966420
22       MRI-ESM2-0 3037320
23            NESM3  966420
24       NorESM2-LM  919800
25       NorESM2-MM 3541230
26      SAM0-UNICON 3541153
27          TaiESM1 3541230
Time difference of 22.80974 secs
CPU times: user 34min 23s, sys: 3min 29s, total: 37min 53s
Wall time: 39min 9s


### Method 2: Arrow Exchange

In [17]:
%%time
%%memit

#df_sub = pd_exchange_df[pd_exchange_df['model'] == "GFDL-ESM4"]
dataset = ds.dataset("figshareairline/combo_data.csv", format="csv")
table = dataset.to_table()
r_table = pyra.converter.py2rpy(table)

5673
rarrow.ChunkedArray: 0.03232526779174805
5673
rarrow.ChunkedArray: 0.02198028564453125
5673
rarrow.ChunkedArray: 0.024966001510620117
5673
rarrow.ChunkedArray: 0.030572175979614258
5673
rarrow.ChunkedArray: 0.02122807502746582
5673
rarrow.ChunkedArray: 0.025564908981323242
5673
rarrow.ChunkedArray: 0.023778915405273438
peak memory: 11076.45 MiB, increment: 3133.72 MiB
CPU times: user 48.7 s, sys: 12.9 s, total: 1min 1s
Wall time: 59.3 s


In [24]:
%%time
%%R -i r_table

start_time <- Sys.time()
result <- r_table %>% collect() %>% count(model)
end_time <- Sys.time()
print(result)

print(end_time - start_time)

# A tibble: 27 x 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# ... with 17 more rows
Time difference of 23.19594 mins
Wall time: 23min 45s


### Method 3: Parquet File

##### Write Time

In [25]:
%%time
%%memit
## writing as a single parquet
pq.write_table(table, 'figshareairline/final.parquet')

peak memory: 6862.77 MiB, increment: 895.64 MiB
Wall time: 11min 56s


##### Read Time

In [ ]:
%%time
%%R
start_time <- Sys.time()
pq_df <- arrow::read_parquet("figshareairline/final.parquet", col_select = c("time", "rain (mm/day)", "model"))
result <- pq_df %>% count(model)
end_time <- Sys.time()
print(result)

print(end_time - start_time)

### Method 4: Feather File

##### Write Time

In [ ]:
%%time
%%memit
# experiment in writing in feather format
feather.write_feather(table, 'figshareairline/final.feather')

##### Read Time

In [ ]:
%%time
%%R

start_time <- Sys.time()
ft_df <- arrow::read_feather("figshareairline/final.feather", col_select = c("time", "rain (mm/day)", "model"))
result <- ft_df %>% count(model)
end_time <- Sys.time()

print(result)

print(end_time - start_time)

### Method Choice and Reasoning

### EDA Visualizations

##### Average Rainfall By Year

In [ ]:
%%R
pq_df_year <- pq_df %>%
    group_by(year=floor_date(time, "year")) %>%
    summarise(year_rain = mean(`rain (mm/day)`))

In [ ]:
%%R
pq_df_year %>%
    ggplot(aes(x=year, y=year_rain)) +
    geom_bar(stat='identity') +
    labs(title="Average Yearly Rainfall (mm/day)",
         x="Year", y="Rainfall (mm/day)")